In [136]:
import pandas as pd 
import numpy as np

data = pd.read_csv('data.csv', header = None)
data.head()

,0,1,2,3,4,5
0,1,19418.0,33212.0,19226.0,7.000,-1
1,2,19418.0,33226.0,19226.0,12.799,1
2,3,19404.0,33240.0,19212.0,12.799,2
3,4,19390.0,33240.0,19198.0,12.799,3
4,5,19362.0,33240.0,19170.0,21.000,4


In [137]:
n = len(data) + 1
parent = np.zeros(n, dtype = int)
rank = np.zeros(n, dtype = int)
for i in range(n):
    parent[i] = i
    
def find(a):
    if parent[a] != a:
        parent[a] = find(parent[a])
    return parent[a]
    
def set_union(a, b):
    ap = find(a)
    bp = find(b)
    if rank[ap] < rank[bp]:
        parent[ap] = bp
    elif rank[ap] > rank[bp]:
        parent[bp] = ap
    else:
        parent[bp] = ap
        rank[ap] += 1

In [138]:
num_rows = 0
for ind, row in data.iterrows():
    src, dest = int(row.iloc[0]), int(row.iloc[5])
    num_rows += 1
    if dest != -1:
        set_union(src, dest)
    else:
        rank[src] = n
num_rows == len(data)

True

In [139]:
comps = {}
for i in range(1, n):
    if parent[i] in comps:
        comps[parent[i]].append(i)
    else:
        comps[parent[i]] = [i]

In [173]:
def build_graph(nodes):
    graph = {}
    for node in nodes:
        src, dest = data.iloc[node - 1, 0], data.iloc[node - 1, 5]
        if dest != -1:
            if dest in graph:
                graph[dest].append(src)
            else:
                graph[dest] = [src]
    return graph

def get_depth_order(graph, root):
    order = []
    queue = [root]
    prev_level_count = 1
    while prev_level_count > 0:
        cur_level_count = 0
        while prev_level_count:
            prev_level_count -= 1
            cur_node = queue.pop(0)
            order.append(cur_node)
            if cur_node in graph:
                cur_level_count += len(graph[cur_node])
                queue += graph[cur_node]
        prev_level_count = cur_level_count
    return order

def branch(root, graph, vis, path):
    cur = root
    while cur:
        path.append(cur)
        vis[cur] = 1
        if cur in graph:
            for child in graph[cur]:
                if not vis[child]:
                    cur = child
                    break
        if cur == path[-1]:
            break
    return path

def branch_driver(root):
    node_list = comps[root]
    graph = build_graph(node_list)
    vis = np.zeros(n, dtype = int)
    branches = []
    depth_order = get_depth_order(graph, root)
    branches.append(branch(root, graph, vis, []))
    num_nodes = len(branches[0])
    for node in depth_order:
        if not vis[node]:
            cur_child = data.iloc[node - 1, 5]
#             print(branch(node, graph, vis, [cur_child]))
            branches.append(branch(node, graph, vis, [cur_child]))
            num_nodes = num_nodes + len(branches[-1]) - 1
    return branches, num_nodes

In [180]:
for key in comps:
    branch_list, num_nodes = branch_driver(key)
    print(f'graph with root {key} has: \n\t\
              {len(comps[key])} total nodes \n\t\
              {len(branch_driver(key))} branches \n\t\
              {num_nodes} nodes accounted for in the branches')

graph with root 1 has: 
	              34190 total nodes 
	              2 branches 
	              34190 nodes accounted for in the branches
graph with root 34191 has: 
	              216 total nodes 
	              2 branches 
	              216 nodes accounted for in the branches
graph with root 34407 has: 
	              305 total nodes 
	              2 branches 
	              305 nodes accounted for in the branches
graph with root 34712 has: 
	              15 total nodes 
	              2 branches 
	              15 nodes accounted for in the branches
graph with root 34727 has: 
	              5 total nodes 
	              2 branches 
	              5 nodes accounted for in the branches


In [176]:
root = 34712
branches, num_nodes = branch_driver(root)
for branch_path in branches:
    print(branch_path)

15 15
[34712, 34713, 34714, 34715, 34716, 34717, 34718, 34719]
[34715, 34724, 34725, 34726]
[34717, 34723]
[34718, 34720, 34721, 34722]
